$\newcommand{\Hset}{\mathcal{H}}$
$\newcommand{\Aset}{\mathcal{A}}$
$\newcommand{\Sset}{\mathcal{S}}$
$\newcommand{\Tset}{\mathcal{T}}$
$\newcommand{\HAinter}{\Hset \cap \Aset}$
$\newcommand{\HAunion}{\Hset \cup \Aset}$
$\newcommand{\Hsize}{\left|\Hset\right|}$
$\newcommand{\Asize}{\left|\Aset\right|}$
$\newcommand{\Tsize}{\left|\Tset\right|}$
$\newcommand{\HAunionsize}{\left|\HAunion\right|}$
$\newcommand{\HAintersize}{\left|\HAinter\right|}$
$\newcommand{\lo}{\mathrm{lo}}$
$\newcommand{\hi}{\mathrm{hi}}$
$\newcommand{\expL}{\mathcal{L}}$
$\newcommand{\expLopt}{\mathcal{L}}$
$\newcommand{\E}{\mathbb{E}}$
$\newcommand{\logit}{\mathrm{logit}}$
$\newcommand{\expit}{\mathrm{expit}}$
$\newcommand{\Var}{\mathrm{Var}}$
$\newcommand{\Cov}{\mathrm{Cov}}$

# Binary Classification Sampling Model — Interactive Simulation

This notebook simulates binary classification with a set of $n$ binary signals,
$$
\Sset = \{ S_1, \dots, S_n \}, \quad S_i \in \{\lo, \hi\}.
$$

A human and an AI each observe subsets of i.i.d. binary signals, denoted $\Hset$ and $\Aset$ respectively. 
Signals in the overlapping set $\Hset \cap \Aset$ are observed by both agents.


## Model

The model has the following parameters:
- $n := |\Sset| \in \mathbb{N}$, the total number of i.i.d. signals
- $p_Y := \Pr (Y=1) \in (0,1)$, the prior probability of $Y\in \{0,1\}$
- $p_0 := \Pr (S_i = \hi \mid Y=0) \in (0,1)$, the likelihood of each $S_i$ when $Y=0$
- $p_1 := \Pr (S_i = \hi \mid Y=1) \in (0,1)$, the likelihood of each $S_i$ when $Y=1$
- $\Hsize := |\Hset| \in \mathbb{N}$, the number of signals observed by the human
- $\Asize := |\Aset| \in \mathbb{N}$, the number of signals observed by the AI
- $\HAintersize := |\Hset \cap \Aset| \in \mathbb{N}$, the number of overlapping signals observed by both agents

The model uses log loss
$$
L(y,d) := -\big( y \log d + (1-y) \log (1-d) \big),
$$
so that when $d$ is the true posterior, the expected loss equals the conditional entropy.

We assume $p_1 > p_0$, non-redundancy, non-degeneracy, each agent is rational (minimizes expected loss given their model), and most importantly, the human is *dependence-neglect* and believes that $\Hset \perp \Aset \mid Y$.


## Signal transformations

For each $\Tset \subseteq \Sset$ (for example, $\Tset$ may be $\Hset$, $\Aset$, or $\HAunion$):

- $$
  \Sigma_\Tset := \sum_{S_i \in \Tset} \mathbf{1} [ S_i=\hi ],
  $$
  the number of high signals in $\Tset$.
- $$
  T := \Pr ( Y=1 \mid \Tset),
  $$
  the calibrated posterior probability estimate when a (rational, correctly specified) agent observes $\Tset$.
- $$
  \ell_T := \log \frac{T}{1-T} = \log \frac{\Pr ( Y=1 \mid \Tset)}{\Pr ( Y=0 \mid \Tset)},
  $$
  the posterior log odds when an agent observes $\Tset$.

Note that all three signal representations, $\Sigma_\Tset$, $T$, and $\ell_T$, are monotonically increasing in each other, and are sufficient statistics for the signals in this model.


## Quantities of interest

We are interested in:

- **Human-only loss**:
  $$
  \expLopt (H) 
  := \mathbb{E} \Big[ \inf_\delta L(Y, \delta(H)) \Big] 
  = \mathbb{E} \big[ L(Y, \Pr (Y=1\mid H)) \big] 
  = H(Y\mid H).
  $$

- **AI-only loss**:
  $$
  \expLopt (A) = H(Y\mid A).
  $$

- **Optimal joint loss**:
  $$
  \expLopt (H,A) = H(Y\mid H,A).
  $$

- **Misspecified joint loss** (under dependence neglect):
  $$
  \expL(\widehat{\delta} (H,A)) 
  = \mathbb{E}\big[ L(Y, \widehat{\delta} (H,A)) \big],
  $$
  where $\widehat{\delta} (H,A)$ is the posterior expectation of $Y$ when assuming $\Hset \perp \Aset \mid Y$.

- **Value of AI signal**:
  $$
  v(A) := \expLopt (\emptyset) - \expLopt (A) 
  = H(Y) - H(Y\mid A) = I(Y;A).
  $$

- **Marginal value of $A$ given $H$**:
  $$
  v(A\mid H) := \expLopt (H) - \expLopt (H,A) 
  = H(Y\mid H) - H(Y\mid H,A) = I(Y;A\mid H).
  $$

- **Candidates for overlap coefficient**:
  * $ \alpha := \frac {\mathrm{Cov}(\ell_H, \ell_A \mid Y)} {\mathrm{Var}(\ell_H \mid Y)}, $ using the same formula as the Gaussian model
  * $ \alpha := \frac {|\HAinter|}{|\Aset|}$, using the same resulting expression as the Gaussian _sampling_ model



## Properties


### Posteriors and decision functions

Let
$$
\logit(x) := \log \frac{x}{1-x}, \quad \ell_Y := \logit(p_Y),
$$
$$
\ell_{\lo} := \log \frac{\Pr(S_i = \lo \mid Y=1)}{\Pr(S_i = \lo \mid Y=0)} 
= \log \frac{1-p_1}{1-p_0},
$$
$$
\ell_{\hi} := \log \frac{\Pr(S_i = \hi \mid Y=1)}{\Pr(S_i = \hi \mid Y=0)} 
= \log \frac{p_1}{p_0}.
$$

Then
$$
\ell_H 
= \ell_Y + \ell_{\lo} \Hsize + (\ell_{\hi} - \ell_{\lo}) \Sigma_H,
$$
$$
\ell_A 
= \ell_Y + \ell_{\lo} \Asize + (\ell_{\hi} - \ell_{\lo}) \Sigma_A.
$$

For the optimal joint posterior based on the union $\HAunion := \Hset \cup \Aset$,
$$
\ell_{H, A} 
= \ell_Y + \ell_{\lo} |\HAunion| + (\ell_{\hi} - \ell_{\lo}) \Sigma_{\HAunion}.
$$

The misspecified joint posterior $\widehat{\delta}(H,A) = \widehat{\Pr}(Y=1\mid H,A)$ is derived from
$$\begin{align*}
\widehat{\ell}_{H, A} 
&= \ell_H + \ell_A - \ell_Y  \\
&= \ell_Y + \ell_{\lo} (\Hsize + \Asize) + (\ell_{\hi} - \ell_{\lo}) (\Sigma_H + \Sigma_A).
\end{align*}
$$

In all cases, the posterior probability is obtained via the logistic function:
$$
\Pr(Y=1\mid \Tset) = \frac{1}{1 + e^{-\ell_\Tset}}.
$$


### Variances, covariances, and overlap

For each $y\in \{0,1\}$, let $p_y := \Pr(S_i=\hi \mid Y=y)$ (so $p_0, p_1$). Then
$$
\Var ( \ell_H \mid Y=y) 
= (\ell_\hi - \ell_\lo)^2 \,\Hsize\, p_y (1-p_y),
$$
$$
\Cov ( \ell_H, \ell_A \mid Y=y) 
= (\ell_\hi - \ell_\lo)^2 \,\HAintersize\, p_y (1-p_y).
$$

Thus, the “candidate” overlap coefficient is
$$
\alpha 
:= \frac{\Cov ( \ell_H, \ell_A \mid Y=y)} {\Var ( \ell_H \mid Y=y)} 
= \frac{\HAintersize}{\Hsize},
\quad \forall y\in \{0,1\}.
$$

Note that $\alpha$ does not depend on $y$.


## Code

In [23]:
import numpy as np
import math
import matplotlib.pyplot as plt

try:
    from ipywidgets import interact, IntSlider, FloatSlider
    import ipywidgets as widgets
    HAS_WIDGETS = True
except ImportError:
    HAS_WIDGETS = False
    print("ipywidgets not installed. Run `pip install ipywidgets` and restart the kernel.")

%matplotlib inline


In [24]:
# Helper functions
 
def logistic(x):
    return 1.0 / (1.0 + np.exp(-x))

def logit(p):
    p = np.clip(p, 1e-15, 1-1e-15)
    return np.log(p / (1 - p))

def binom_pmf(k, n, p):
    """Binomial pmf: P(K=k) for K~Bin(n,p)."""
    if k < 0 or k > n:
        return 0.0
    return math.comb(n, k) * (p**k) * ((1-p)**(n-k))

def log_loss(y, t, eps=1e-15):
    """Binary log loss (negative log-likelihood)."""
    t = np.clip(t, eps, 1-eps)
    return - (y * np.log(t) + (1-y) * np.log(1-t))

### Defunct: Monte Carlo

The code blocks below simulate by generating random signals, which we didn't end up using.

In [25]:
# Cell 2: simulate one trial
def simulate_one_trial(n_H, n_A, n_both, p_Y, p0, p1, rng=None):
    if rng is None:
        rng = np.random.default_rng()

    n_both = min(n_both, n_H, n_A)
    n_H_only = max(n_H - n_both, 0)
    n_A_only = max(n_A - n_both, 0)

    # sample label
    y = rng.binomial(1, p_Y)
    p_y = p1 if y == 1 else p0

    ell_Y = logit(p_Y)
    ell_hi = np.log(p1 / p0)
    ell_lo = np.log((1 - p1) / (1 - p0))

    # sample hi count for each signal group
    hi_both   = rng.binomial(n_both,   p_y)
    hi_H_only = rng.binomial(n_H_only, p_y)
    hi_A_only = rng.binomial(n_A_only, p_y)

    # H-only posterior
    hi_H = hi_both + hi_H_only
    n_H_total = n_both + n_H_only
    T_H = compute_posterior_from_counts(hi_H, n_H_total, ell_Y, ell_hi, ell_lo) if n_H_total > 0 else p_Y

    # A-only posterior
    hi_A = hi_both + hi_A_only
    n_A_total = n_both + n_A_only
    T_A = compute_posterior_from_counts(hi_A, n_A_total, ell_Y, ell_hi, ell_lo) if n_A_total > 0 else p_Y

    # joint posterior
    hi_joint = hi_both + hi_H_only + hi_A_only
    n_joint = n_both + n_H_only + n_A_only
    T_joint = compute_posterior_from_counts(hi_joint, n_joint, ell_Y, ell_hi, ell_lo) if n_joint > 0 else p_Y

    return y, p_Y, T_H, T_A, T_joint

In [26]:
# Cell 3: expected log loss estimator
def log_loss(y, t, eps=1e-12):
    t = np.clip(t, eps, 1 - eps)
    return - (y * np.log(t) + (1 - y) * np.log(1 - t))

def estimate_losses(
    n_H, n_A, n_both,
    p_Y=0.5, p0=0.2, p1=0.8,
    n_trials=20000, seed=None
):
    rng = np.random.default_rng(seed)

    L_prior = 0
    L_H = 0
    L_A = 0
    L_joint = 0

    for _ in range(n_trials):
        y, T_prior, T_H, T_A, T_joint = simulate_one_trial(
            n_H, n_A, n_both, p_Y, p0, p1, rng=rng
        )

        L_prior += log_loss(y, T_prior)
        L_H     += log_loss(y, T_H)
        L_A     += log_loss(y, T_A)
        L_joint += log_loss(y, T_joint)

    return {
        "prior": L_prior / n_trials,
        "H":     L_H     / n_trials,
        "A":     L_A     / n_trials,
        "joint": L_joint / n_trials
    }

In [27]:
# Cell 4: loss plotting
def plot_losses(
    n_H=10, n_A=10, n_both=5,
    p_Y=0.5, p0=0.3, p1=0.7,
    n_trials=20000, seed=0
):
    n_both = min(n_both, n_H, n_A)
    results = estimate_losses(n_H, n_A, n_both, p_Y, p0, p1, n_trials, seed)

    labels = ["prior", "H", "A", "joint"]
    values = [results[k] for k in labels]

    plt.figure(figsize=(6,4))
    bars = plt.bar(labels, values)
    plt.ylabel("Expected log loss")
    plt.title(
        f"p_Y={p_Y:.2f}, p0={p0:.2f}, p1={p1:.2f}, |H|={n_H}, |A|={n_A}, |H∩A|={n_both}"
    )

    for b, v in zip(bars, values):
        plt.text(b.get_x() + b.get_width()/2, v, f"{v:.3f}",
                 ha='center', va='bottom', fontsize=9)

    plt.ylim(0, max(values) * 1.1)
    plt.show()

    return results

In [28]:
# Cell 6: interactive sliders (ipywidgets)
if HAS_WIDGETS:
    interact(
        plot_losses,
        n_H=IntSlider(description="|H|", min=0, max=50, step=1, value=10),
        n_A=IntSlider(description="|A|", min=0, max=50, step=1, value=10),
        n_both=IntSlider(description="|H∩A|", min=0, max=50, step=1, value=5),
        p_Y=FloatSlider(description="p_Y", min=0.01, max=0.99, step=0.01, value=0.5),
        p0=FloatSlider(description="p0", min=0.01, max=0.99, step=0.01, value=0.3),
        p1=FloatSlider(description="p1", min=0.01, max=0.99, step=0.01, value=0.7),
        n_trials=IntSlider(description="n_trials", min=1000, max=50000, step=1000, value=10000),
        seed=IntSlider(description="seed", min=0, max=9999, step=1, value=0)
    );
else:
    print("ipywidgets not available; install it to use sliders.")

interactive(children=(IntSlider(value=10, description='|H|', max=50), IntSlider(value=10, description='|A|', m…

### New: Compute distributional parameters directly

In [29]:
def compute_exact_losses(
    n_H, n_A, n_both,
    p_Y=0.5, p0=0.3, p1=0.7
):
    """
    Compute expected losses exactly (via finite sums over binomial counts):
      - prior-only loss H(Y)
      - human-only H(Y|H)
      - AI-only H(Y|A)
      - optimal joint H(Y|H,A)
      - misspecified joint loss under dependence neglect
    
    Returns a dict of losses and some derived values.
    """
    # sanitize overlap
    n_both = min(n_both, n_H, n_A)
    n_H_only = max(n_H - n_both, 0)
    n_A_only = max(n_A - n_both, 0)
    
    # prior log-odds and per-signal log-likelihood ratios
    ell_Y  = logit(p_Y)
    ell_hi = np.log(p1 / p0)
    ell_lo = np.log((1-p1) / (1-p0))
    
    # union size
    n_union = n_both + n_H_only + n_A_only
    n_H_tot = n_both + n_H_only
    n_A_tot = n_both + n_A_only
    
    # For the true model, expected prior loss = H(Y)
    prior_loss = - (p_Y * np.log(p_Y) + (1-p_Y) * np.log(1-p_Y))
    
    # Initialize expectations
    loss_H      = 0.0  # H(Y|H) when using true posterior given H
    loss_A      = 0.0  # H(Y|A)
    loss_joint  = 0.0  # H(Y|H,A) using optimal posterior given union
    loss_mis    = 0.0  # expected loss using mis-specified posterior (dependence neglect)
    
    # Sum over Y in {0,1} and counts in the three groups
    for y in (0, 1):
        w_y = p_Y if y == 1 else (1 - p_Y)
        p_sig = p1 if y == 1 else p0  # P(S=1 | Y=y)
        
        # Precompute binomial pmfs for each group given Y
        pmf_both = [binom_pmf(k, n_both,   p_sig) for k in range(n_both+1)]
        pmf_Ho   = [binom_pmf(k, n_H_only, p_sig) for k in range(n_H_only+1)]
        pmf_Ao   = [binom_pmf(k, n_A_only, p_sig) for k in range(n_A_only+1)]
        
        for k_b in range(n_both+1):
            p_b = pmf_both[k_b]
            for k_Ho in range(n_H_only+1):
                p_Ho = pmf_Ho[k_Ho]
                for k_Ao in range(n_A_only+1):
                    p_Ao = pmf_Ao[k_Ao]
                    
                    prob = w_y * p_b * p_Ho * p_Ao
                    if prob == 0.0:
                        continue
                    
                    # Counts seen by H, A, and union
                    sig_H = k_b + k_Ho
                    sig_A = k_b + k_Ao
                    sig_U = k_b + k_Ho + k_Ao  # union
                    
                    # log-odds and posteriors
                    # H-only
                    ell_H = ell_Y + n_H_tot * ell_lo + sig_H * (ell_hi - ell_lo)
                    T_H   = logistic(ell_H)
                    
                    # A-only
                    ell_A = ell_Y + n_A_tot * ell_lo + sig_A * (ell_hi - ell_lo)
                    T_A   = logistic(ell_A)
                    
                    # optimal joint: based on union
                    ell_U = ell_Y + n_union * ell_lo + sig_U * (ell_hi - ell_lo)
                    T_U   = logistic(ell_U)
                    
                    # mis-specified joint (dependence neglect)
                    ell_hat = ell_H + ell_A - ell_Y
                    T_hat   = logistic(ell_hat)
                    
                    # accumulate losses
                    loss_H     += prob * log_loss(y, T_H)
                    loss_A     += prob * log_loss(y, T_A)
                    loss_joint += prob * log_loss(y, T_U)
                    loss_mis   += prob * log_loss(y, T_hat)
    
    # Information-theoretic derived quantities
    v_A      = prior_loss - loss_A        # value of A alone
    v_A_given_H = loss_H - loss_joint     # marginal value of A given H
    overlap_alpha = (n_both / n_H) if n_H > 0 else np.nan
    
    return {
        "H(Y)": prior_loss,
        "H(Y|H)": loss_H,
        "H(Y|A)": loss_A,
        "H(Y|H,A)": loss_joint,
        "E[loss_misspecified]": loss_mis,
        "v(A)": v_A,
        "v(A|H)": v_A_given_H,
        "alpha_candidate": overlap_alpha,
        "n_H": n_H,
        "n_A": n_A,
        "n_both": n_both
    }


Quick test (non-interactive)

In [ ]:
res_test = compute_exact_losses(
    n_H=10, n_A=15, n_both=5,
    p_Y=0.5, p0=0.2, p1=0.8
)
res_test

{'H(Y)': np.float64(0.6931471805599453),
 'H(Y|H)': np.float64(0.04513945272073848),
 'H(Y|A)': np.float64(0.013050227549253505),
 'H(Y|H,A)': np.float64(0.0038776516658284503),
 'E[loss_misspecified]': np.float64(0.007553455329953645),
 'v(A)': np.float64(0.6800969530106917),
 'v(A|H)': np.float64(0.04126180105491003),
 'alpha_candidate': 0.5,
 'n_H': 10,
 'n_A': 15,
 'n_both': 5}

### Basic bar chart

In [ ]:
def plot_losses_exact(
    n_H=10, n_A=10, n_both=5,
    p_Y=0.5, p0=0.3, p1=0.7
):
    results = compute_exact_losses(n_H, n_A, n_both, p_Y, p0, p1)
    
    labels = ["H(Y)", "H(Y|H)", "H(Y|A)", "H(Y|H,A)", "misspec"]
    values = [
        results["H(Y)"],
        results["H(Y|H)"],
        results["H(Y|A)"],
        results["H(Y|H,A)"],
        results["E[loss_misspecified]"],
    ]
    
    plt.figure(figsize=(7,4))
    bars = plt.bar(labels, values)
    plt.ylabel("Expected log loss (nats)")
    plt.title(
        f"p_Y={p_Y:.2f}, p0={p0:.2f}, p1={p1:.2f}, "
        f"|H|={n_H}, |A|={n_A}, |H∩A|={min(n_both, n_H, n_A)}"
    )
    for b, v in zip(bars, values):
        plt.text(b.get_x() + b.get_width()/2, v, f"{v:.3f}",
                 ha="center", va="bottom", fontsize=9)
    plt.ylim(0, max(values) * 1.15)
    plt.show()
    
    # print("alpha candidate (Cov/Var) =", results["alpha_candidate"])
    # print("v(A)      = H(Y) - H(Y|A)      =", results["v(A)"])
    # print("v(A|H)    = H(Y|H) - H(Y|H,A)  =", results["v(A|H)"])
    # print("H(Y|H,A) optimal joint        =", results["H(Y|H,A)"])
    # print("Misspecified joint loss       =", results["E[loss_misspecified]"])
    
    return results

if HAS_WIDGETS:
    interact(
        plot_losses_exact,
        n_H=IntSlider(description="|H|", min=0, max=40, step=1, value=10),
        n_A=IntSlider(description="|A|", min=0, max=40, step=1, value=10),
        n_both=IntSlider(description="|H∩A|", min=0, max=40, step=1, value=5),
        p_Y=FloatSlider(description="p_Y", min=0.01, max=0.99, step=0.01, value=0.5),
        p0=FloatSlider(description="p0",  min=0.01, max=0.99, step=0.01, value=0.3),
        p1=FloatSlider(description="p1",  min=0.01, max=0.99, step=0.01, value=0.7),
    );
else:
    print("ipywidgets not available; install it to use sliders.")


interactive(children=(IntSlider(value=10, description='|H|', max=40), IntSlider(value=10, description='|A|', m…

### Loss vs. $|\Aset|$

In [ ]:
def sweep_over_A(
    n_H=10,
    n_both_base=5,
    p_Y=0.5,
    p0=0.3,
    p1=0.7,
    max_A=30
):
    """
    Fix n_H, nominal n_both (overlap), and p_Y, p0, p1.
    Plot how losses change as |A| varies from 0 to max_A.
    
    Note: for each |A|, the *effective* overlap is min(n_both_base, n_H, |A|).
    """
    A_values = list(range(0, max_A + 1))
    loss_H_vals      = []
    loss_A_vals      = []
    loss_joint_vals  = []
    loss_mis_vals    = []
    
    for n_A in A_values:
        # effective overlap
        n_both_eff = min(n_both_base, n_H, n_A)
        res = compute_exact_losses(
            n_H=n_H,
            n_A=n_A,
            n_both=n_both_eff,
            p_Y=p_Y,
            p0=p0,
            p1=p1
        )
        loss_H_vals.append(res["H(Y|H)"])
        loss_A_vals.append(res["H(Y|A)"])
        loss_joint_vals.append(res["H(Y|H,A)"])
        loss_mis_vals.append(res["E[loss_misspecified]"])
    
    plt.figure(figsize=(7,4))
    plt.plot(A_values, loss_H_vals,     label="H-only loss H(Y|H)")
    plt.plot(A_values, loss_A_vals,     label="A-only loss H(Y|A)")
    plt.plot(A_values, loss_joint_vals, label="Optimal joint H(Y|H,A)")
    plt.plot(A_values, loss_mis_vals,   label="Misspecified joint loss")
    
    plt.xlabel("|A|")
    plt.ylabel("Expected log loss (nats)")
    plt.title(
        f"Loss vs |A| (|H|={n_H}, nominal |H∩A|={n_both_base}, "
        f"p_Y={p_Y:.2f}, p0={p0:.2f}, p1={p1:.2f})"
    )
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()


# Interactive widget for sweep over |A|
if HAS_WIDGETS:
    interact(
        sweep_over_A,
        n_H=IntSlider(description="|H|", min=0, max=40, step=1, value=10),
        n_both_base=IntSlider(description="nom. |H∩A|", min=0, max=40, step=1, value=5),
        p_Y=FloatSlider(description="p_Y", min=0.01, max=0.99, step=0.01, value=0.5),
        p0=FloatSlider(description="p0",  min=0.01, max=0.99, step=0.01, value=0.3),
        p1=FloatSlider(description="p1",  min=0.01, max=0.99, step=0.01, value=0.7),
    );
else:
    print("ipywidgets not available; install it to use the sliders.")


interactive(children=(IntSlider(value=10, description='|H|', max=40), IntSlider(value=5, description='|H∩A|', …

### Loss vs. $|\HAunion|$

In [37]:
def sweep_over_overlap(
    n_H=10,
    n_A=10,
    p_Y=0.5,
    p0=0.3,
    p1=0.7
):
    """
    Fix n_H, n_A, p_Y, p0, p1.
    Plot how losses change as |H∩A| varies from 0 to min(|H|,|A|).
    """
    max_both = min(n_H, n_A)
    both_values = list(range(0, max_both + 1))
    
    loss_H_vals      = []
    loss_A_vals      = []
    loss_joint_vals  = []
    loss_mis_vals    = []
    
    for n_both in both_values:
        res = compute_exact_losses(
            n_H=n_H,
            n_A=n_A,
            n_both=n_both,
            p_Y=p_Y,
            p0=p0,
            p1=p1
        )
        loss_H_vals.append(res["H(Y|H)"])
        loss_A_vals.append(res["H(Y|A)"])
        loss_joint_vals.append(res["H(Y|H,A)"])
        loss_mis_vals.append(res["E[loss_misspecified]"])
    
    plt.figure(figsize=(7,4))
    plt.plot(both_values, loss_H_vals,     label="H-only loss H(Y|H)")
    plt.plot(both_values, loss_A_vals,     label="A-only loss H(Y|A)")
    plt.plot(both_values, loss_joint_vals, label="Optimal joint H(Y|H,A)")
    plt.plot(both_values, loss_mis_vals,   label="Misspecified joint loss")
    
    plt.xlabel("|H∩A|")
    plt.ylabel("Expected log loss (nats)")
    plt.title(
        f"Loss vs |H∩A| (|H|={n_H}, |A|={n_A}, "
        f"p_Y={p_Y:.2f}, p0={p0:.2f}, p1={p1:.2f})"
    )
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()


# Interactive widget for sweep over |H∩A|
if HAS_WIDGETS:
    interact(
        sweep_over_overlap,
        n_H=IntSlider(description="|H|", min=0, max=40, step=1, value=10),
        n_A=IntSlider(description="|A|", min=0, max=40, step=1, value=10),
        p_Y=FloatSlider(description="p_Y", min=0.01, max=0.99, step=0.01, value=0.5),
        p0=FloatSlider(description="p0",  min=0.01, max=0.99, step=0.01, value=0.3),
        p1=FloatSlider(description="p1",  min=0.01, max=0.99, step=0.01, value=0.7),
    );
else:
    print("ipywidgets not available; install it to use the sliders.")


interactive(children=(IntSlider(value=10, description='|H|', max=40), IntSlider(value=10, description='|A|', m…

### 2D phase diagram: $|\Aset|$ (y-axis) vs. $|\HAunion|$ (x-axis)

Y-axis $|\Aset|$ is monotone with signal quality, so greater y-axis means better AI signal.

Greater x-axis means more overlap.

Since $|\Hset|$ is fixed for this analysis, this is equivalent to plotting $\alpha = \Cov (\ell_H, \ell_A | Y) / \Var (\ell_H | Y)$ as the x-axis (just that the x-axis would then range from 0 to 1).

In [41]:
from matplotlib.colors import ListedColormap
from ipywidgets import interact, IntSlider, FloatSlider

def classify_point(n_H, n_A, n_both, p_Y, p0, p1):
    """
    For a given (n_H, n_A, n_both, p_Y, p0, p1), classify which regime we are in:
      0 = complementarity (misspecified joint < min(H-only, A-only))
      1 = human alone best
      2 = AI alone best
    Returns (category, losses_dict).
    """
    res = compute_exact_losses(n_H=n_H, n_A=n_A, n_both=n_both,
                               p_Y=p_Y, p0=p0, p1=p1)
    loss_H   = res["H(Y|H)"]
    loss_A   = res["H(Y|A)"]
    loss_mis = res["E[loss_misspecified]"]
    
    # Complementarity: misspecified joint strictly better than either alone
    if loss_mis < min(loss_H, loss_A):
        cat = 0
    # Human alone best
    elif loss_H <= loss_A and loss_H <= loss_mis:
        cat = 1
    # AI alone best
    elif loss_A <= loss_H and loss_A <= loss_mis:
        cat = 2
    else:
        # Fallback for strange tie cases
        cat = 1
    return cat, res


def plot_region_HA_grid(
    n_H=10,
    p_Y=0.5,
    p0=0.3,
    p1=0.7,
    max_A=40
):
    """
    X-axis: |H ∩ A|
    Y-axis: |A|
    Fixed parameters: |H|, p_Y, p0, p1.
    
    Each point (|H∩A|, |A|) is colored by:
      - Complementarity (misspec joint < both H-only and A-only)
      - Human alone best
      - AI alone best
    """
    # X: overlap size from 0..n_H
    max_both = n_H
    x_vals = list(range(0, max_both + 1))   # |H∩A|
    # Y: A size from 0..max_A
    y_vals = list(range(0, max_A + 1))      # |A|
    
    # data[y, x] = category in {0,1,2}, or -1 for invalid (overlap > |A| or > |H|)
    data = -1 * np.ones((len(y_vals), len(x_vals)), dtype=int)
    
    for iy, n_A in enumerate(y_vals):
        for ix, n_both in enumerate(x_vals):
            # invalid states: overlap can't exceed |H| or |A|
            if n_both > n_H or n_both > n_A:
                continue
            cat, _ = classify_point(n_H=n_H, n_A=n_A, n_both=n_both,
                                    p_Y=p_Y, p0=p0, p1=p1)
            data[iy, ix] = cat
    
    # Create a masked array so invalid points show up as blank
    masked = np.ma.masked_where(data < 0, data)
    
    # colormap for 3 categories: 0=complementarity, 1=human, 2=AI
    cmap = ListedColormap(["tab:green", "tab:blue", "tab:orange"])
    
    plt.figure(figsize=(7, 6))
    im = plt.imshow(
        masked,
        origin="lower",
        aspect="auto",
        cmap=cmap,
        extent=[x_vals[0] - 0.5, x_vals[-1] + 0.5, y_vals[0] - 0.5, y_vals[-1] + 0.5]
    )
    plt.colorbar(
        im,
        ticks=[0, 1, 2],
        label="Regime"
    )
    
    # Relabel colorbar ticks
    cbar = plt.gci().colorbar
    cbar.ax.set_yticklabels([
        "Complementarity\n(misspec joint best)",
        "Human alone best",
        "AI alone best"
    ])
    
    plt.xlabel("|H∩A|")
    plt.ylabel("|A|")
    plt.title(
        f"Best performer by (|H∩A|, |A|)\n"
        f"|H|={n_H}, p_Y={p_Y:.2f}, p0={p0:.2f}, p1={p1:.2f}"
    )
    plt.grid(False)  # grid looks messy on imshow; leave off
    
    # Overlay valid region boundary (|H∩A| ≤ min(|H|, |A|))
    # Draw the line |H∩A| = |A| and |H∩A| = |H| for visualization.
    # Only the intersection of |H∩A|≤|H| and |H∩A|≤|A| is valid.
    # We'll simply outline the triangle where n_both <= min(n_H, n_A) visually:
    xs = []
    ys = []
    for n_A in y_vals:
        xs.append(min(n_A, n_H))
        ys.append(n_A)
    plt.plot(xs, ys, color="k", linestyle="--", linewidth=1, alpha=0.7)
    
    plt.show()


# Interactive widget
if HAS_WIDGETS:
    interact(
        plot_region_HA_grid,
        n_H=IntSlider(description="|H|", min=0, max=40, step=1, value=20),
        p_Y=FloatSlider(description="p_Y", min=0.01, max=0.99, step=0.01, value=0.5),
        p0=FloatSlider(description="p0",  min=0.01, max=0.99, step=0.01, value=0.3),
        p1=FloatSlider(description="p1",  min=0.01, max=0.99, step=0.01, value=0.7),
        # max_A is fixed internally to 40, but you can expose it as a slider if you like.
    );
else:
    print("ipywidgets not available; install it to use the sliders.")


interactive(children=(IntSlider(value=20, description='|H|', max=40), FloatSlider(value=0.5, description='p_Y'…

### 2D phase diagram: $|\Aset|$ (y-axis) vs. $\frac{|\HAunion|}{|\Aset|}$ (x-axis)

The x-axis uses the same resulting expression as the Gaussian _sampling_ model.

In [42]:
def plot_region_overlap_ratio_scatter(
    n_H=20,
    p_Y=0.5,
    p0=0.3,
    p1=0.7,
    max_A=40
):
    """
    Scatter version of the HA regime plot.

    Underlying data:
      - Fix |H|, p_Y, p0, p1.
      - Enumerate over all integer pairs (|A|, |H∩A|) with
          0 <= |A| <= max_A,
          0 <= |H∩A| <= min(|H|, |A|).

    Plot:
      - X-axis: overlap ratio r = |H ∩ A| / |A|
      - Y-axis: |A|
      - Each valid (|A|, |H∩A|) shown as a scatter point.
      - Color encodes regime:
          0 = complementarity (misspec joint best)
          1 = human alone best
          2 = AI alone best

    Note: the degenerate case |A|=0 is omitted, since r is undefined (0/0).
    """
    xs = []
    ys = []
    cats = []

    for n_A in range(0, max_A + 1):
        # skip n_A = 0 because |H∩A| / |A| is undefined
        if n_A == 0:
            continue
        for n_both in range(0, min(n_A, n_H) + 1):
            cat, _ = classify_point(
                n_H=n_H,
                n_A=n_A,
                n_both=n_both,
                p_Y=p_Y,
                p0=p0,
                p1=p1,
            )
            ratio = n_both / n_A  # |H∩A| / |A|

            xs.append(ratio)
            ys.append(n_A)
            cats.append(cat)

    xs = np.array(xs)
    ys = np.array(ys)
    cats = np.array(cats)

    # Colors/labels for the three regimes
    colors = {
        0: "tab:green",   # complementarity
        1: "tab:blue",    # human alone best
        2: "tab:orange",  # AI alone best
    }
    labels = {
        0: "Complementarity\n(misspec joint best)",
        1: "Human alone best",
        2: "AI alone best",
    }

    plt.figure(figsize=(7, 6))

    # Plot each category separately for legend labels
    for cat in [0, 1, 2]:
        mask = (cats == cat)
        if not np.any(mask):
            continue
        plt.scatter(
            xs[mask],
            ys[mask],
            s=30,
            alpha=0.8,
            color=colors[cat],
            label=labels[cat],
        )

    plt.xlabel(r"|H ∩ A| / |A|")
    plt.ylabel(r"|A|")
    plt.xlim(-0.05, 1.05)
    plt.ylim(0, max_A + 1)
    plt.title(
        "Best performer by overlap ratio and |A|\n"
        f"|H|={n_H}, p_Y={p_Y:.2f}, p0={p0:.2f}, p1={p1:.2f}"
    )
    plt.grid(True, linestyle=":", alpha=0.4)
    plt.legend(loc="best", fontsize=9, frameon=True)
    plt.show()


# Interactive widget
if HAS_WIDGETS:
    interact(
        plot_region_overlap_ratio_scatter,
        n_H=IntSlider(description="|H|", min=0, max=40, step=1, value=20),
        p_Y=FloatSlider(description="p_Y", min=0.01, max=0.99, step=0.01, value=0.5),
        p0=FloatSlider(description="p0",  min=0.01, max=0.99, step=0.01, value=0.3),
        p1=FloatSlider(description="p1",  min=0.01, max=0.99, step=0.01, value=0.7),
        # max_A is fixed internally to 40, but you can expose it as a slider if you like.
    );
else:
    print("ipywidgets not available; install it to use the sliders.")


interactive(children=(IntSlider(value=20, description='|H|', max=40), FloatSlider(value=0.5, description='p_Y'…